In [1]:
#Extract (from only Sloane records), extract all catnum where place or person name (or both) mentioned with 
#page numbers<pb>, <div2>, and text within <div3>
from lxml import etree
import re
import csv

In [2]:
path = '/Users/deborahleem/Documents/PhD/SloaneDBwork19/Miscellanies5Ah_MASTER_fixed_whitespaceDL.xml'
doc = etree.parse(open(path))
root = doc.getroot()

In [3]:
part_excluded = {
  "EPF108491027v": ["2108","2109","2110","2111"], # "Miscellanies"
  "EPF108491177v": ["1","2"], # "Antiquities"    
}

excluded = {
  "EPF108491178v": ["1","2","3","4"], # "Antiquities"
  "EPF108491179v": ["4","5","6","7","8","1"], # "Antiquities"
  "EPF108491180v": ["2","3"], # "Antiquities"
  "EPF108491181v": ["4","5"], # "Antiquities"
  "EPF108491182v": ["1","2","3","4","5"], # "Antiquities"
  "EPF108491183v": ["6","7","8","9","10","11","12"], # "Antiquities"
  "EPF108491184v": ["13","14","1","2","3","4","5","1"], # "Antiquities"
  "EPF108491186v": ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"], # "Antiquities"
  "EPF108491187v": ["20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39"], # "Antiquities"
  "EPF108491188v": ["40","41","42","43","44","45","46","47","48","1","2","3","4","5","6","7","8","9"], # "Antiquities"
  "EPF108491189v": ["10","11","12","1","2","3","4","5","6"], # "Antiquities"
  "EPF108491190v": ["7","8","9","10","11"], # "Antiquities"
}


In [4]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return label.text
            return
        element = element.getparent()

In [5]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()

In [6]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

In [7]:
#ignore non-Sloane catnums and index <label type="index">. End of index </div2>


In [10]:
people_csv_data = []

for div3 in root.iter("{*}div3"):
    persons = list(div3.iter('{*}persName'))
    places = list(div3.iter('{*}placeName'))
    catnums = list(div3.iter('{*}catnum'))
    clean_catnums = [cleanup_catnum(''.join(catnum.itertext())) for catnum in catnums]
    
    page_id = find_pb_xml_id(div3)
    label = find_div2_label(div3)
    
    if label == 'Index.':
        continue
    
    if len(persons) > 0 or len(places) > 0:
        if page_id in excluded:
            continue

        people_csv_data.append(["div3"])
        print("div3")

        people_csv_data.append(["div2 label", label])
        print(label)

        people_csv_data.append(["page id", page_id])
        print(page_id)
        
        div3_text = ''.join(div3.itertext())
        people_csv_data.append(["div3 text", div3_text])
        print(div3_text)

#         if page_id in excluded:
#             continue
#             print('maybe exclude this all', page_id)
        
        for person in persons:
            name = re.sub(r'\s+', " ", ''.join(person.itertext()))
            name = re.sub(r'^\s+', '', name)
            name = re.sub(r'\s+$', '', name)            
            print(name)
            people_csv_data.append(["person", name])

        for place in places:
            name = re.sub(r'\s+', " ", ''.join(place.itertext()))
            name = re.sub(r'^\s+', '', name)
            name = re.sub(r'\s+$', '', name)            
            print(name)
            people_csv_data.append(["place", name])

        for name in clean_catnums:
            print(name)
            people_csv_data.append(["catnum", name])
        print("")

div3
Miscellanies.
EPF108491002v

            -1799.
             A bow said to be made of a bone from East Indies given to me by Mr. Amyand.
            
            
              252.
            
          
Mr. Amyand.
East Indies
1799

div3
Miscellanies.
EPF108491002v

            -1801.
             A trumpet from Norway made of 2 pieces of wood about 5 foot
              long in form of a trumpet fastened by birch tree
              bark from Mr. Theobalds.
            
              245.
            
          
Mr. Theobalds.
Norway
1801

div3
Miscellanies.
EPF108491002v

            -1806.
            Large severall figured pieces of blew
              glasse taken from the bottom of the
              fire place of a furnace given me by Mr. Roberts.
            
              216 b
              30 C
            
          
Mr. Roberts
1806

div3
Miscellanies.
EPF108491002v

            -1807.
            A leather boracho or bottle wherein is carried wine on a
              j

Miscellanies.
EPF108491017v

            
              2037.
            
            
              This was some Indian Instru= ment ye hole drilld th o
                row it is done very true smooth
                & even I suppose they fixed a litle flint in the end of
                a stick by which they drilled ye hole tho row I found
                it in this shape so you may guess as well as I have it was made & what
                use it was desighned for from  Dr.
                  Bartram.
            
            
              199 c c
              28. b.
            
          
Dr. Bartram
2037

div3
Miscellanies.
EPF108491018

            -218.
            A candle made of the berries of
              a tree from Carolina.
          
Carolina
218

div3
Miscellanies.
EPF108491018

            219.
            A China
              chapstick of ivory
               cutt &
              painted over.  Quaitts  vid du Hald. p. 17. tom. 2
            
              162
 

530

div3
Miscellanies.
EPF108491049

            -531.
            Sope from Joppa. P.
            
              99.
            
          
Joppa
531

div3
Miscellanies.
EPF108491049

            -532.
            Sope of Melo Island. P.
          
Melo Island
532

div3
Miscellanies.
EPF108491050

            -539.
            An East India
              cane
              lanthorne --- 0. 5. 0.
            
              245.
            
          
East India
539

div3
Miscellanies.
EPF108491050

            -540.
            A leather
              purse embroidered together with the figure of a hand
              in horn? fastened to it to be worn to prevent thunder in the mines of Sta. Maria near
                Campeche in New Spain taken from
                the
              Indians by the English of Jamaica from the Spainyards when they
              took those mines & given me by Coll.
                  Laws.
            
              199. b
              29 C
           

Miscellanies.
EPF108491093

            -870.
            The drum stick both given me by Mr. Heathcote.
            
              b 245.
            
          
Mr. Heathcote
870

div3
Miscellanies.
EPF108491094

            -873.
            A scymiter handle of 
              wood with a lyons head carved upon it. from Dr. Waldo ---} 0. 2. 6.
            
              216. a
              27.
            
          
Dr. Waldo
873

div3
Miscellanies.
EPF108491094

            -874.
            A Syringe for injecting the vessells wt. wax of severall colours?
              or From Dr. Lavater.
            
              253. <
            
          
Dr. Lavater
874

div3
Miscellanies.
EPF108491094

            -876.
            A purse made of an herb from guinea. 'Tis red & gray
               coloured. Given me by Mr. Burnett.
            
              244.
              19 W
            
          
Mr. Burnett
876

div3
Miscellanies.
EPF108491094

            -877.
            

EPF108491122

            -1246.
            Cinders from the iron furnaces in
                Sussex by  Mr.
                Fuller ; glasse botles
              are made of it at Bristoll!
            
              99
            
          
Mr. Fuller
Sussex
Bristoll
1246

div3
Miscellanies.
EPF108491123

            -1247. 5
            A case of instruments from
                China for the ears
                wt. the instrument.
            
              199 C a
              25
            
            X
          
China
1247 5

div3
Miscellanies.
EPF108491123

            -1248.
            a breast Jewell made in America by the Indians of beetles
                  wings sett upon straw of a gold colour.
            
              < 199 e
              24.
            
          
America
1248

div3
Miscellanies.
EPF108491123

            -1250
            A snow shoe from Hudsons bay.
          
Hudsons bay
1250

div3
Miscellanies.
EPF108491123

            -1257.
         

Smyrna
1539

div3
Miscellanies.
EPF108491137

            -1540.
            Turkish
              papooches & Terleaks which ladies wear within
              doors at home at Constantinople &
              Smyrna. Id. S. B
            
              < 226.
              20 J.
            
            X
          
Turkish
Constantinople
Smyrna
1540

div3
Miscellanies.
EPF108491137

            -1542.
            An Indian deitie from China. Mr. Maidston.
            
              f. < 245.
              19. f
            
          
Mr. Maidston
China
1542

div3
Miscellanies.
EPF108491137

            -1543.
            Henry the 4th.. of France carv'd on a shell in form of a Centaur.
            
              22 4 b. 223
              AWF
              ✓
              X
            
          
France
1543

div3
Miscellanies.
EPF108491137

            -1544.
            A Thunderbolt from Bohemia. A wedge or antient weapon of granite stone.
              Vid
              
          

EPF108491152

            -1798.
            Part of a Hayrick burnt in France & lighting from Mr. Sequier.
            
              99
            
          
Mr. Sequier
France
1798

div3
Antiquities.
EPF108491153v

            -700.
            The figure of an old bearded man in
                terra cotta
              Ægyptian from
               Cardinall Gualtieris collection.
                Vid. 287.
            
              216. m
            
          
Cardinall Gualtieris
700

div3
Antiquities.
EPF108491153v

            -726.
            An earthen
              bottle of gray earth colourd
                black in the form of a porpoise wt. a handle from Peru by  Dr. Houston said to be one of their gods. (
                  idol.)
            
              A.W.F. 1882
              242
            
          
Dr. Houston
Peru
726

div3
Antiquities.
EPF108491154

            -4.
            A Gray
              earthen
              bottle taken up by
              

Antiquities.
EPF108491182

            -269.
            A sepulchral Roman lamp wt.
              protuberancies- on it. From Rome.
            
              E H 250.
              X
            
          
Rome
269

div3
Antiquities.
EPF108491183

            -297.
            A brasse Roman ring wt. a key from Rome.
              Vid.
                Montfaucon LIV pl. a la 106 page t. 3.
            
              216. o
            
          
Rome.
297

div3
Antiquities.
EPF108491183

            -300.
            A wedge wt. a loop, of
                brasse taken up with in a mile & an
                halfe of Windsor in a
              common five foot deep where was a melting house formerly & where are dug
              up many potts of an unusuall figure. 
                Vid
                Montfaucon tom 3. p: 2 pl. CLXXXVIII. p. 349 329
                
              L. 4in
            
              E H.
              A.W.F. 1852. A.
              X
            
       

EPF108491230

            247.
            The Duke of Bavaria ? on horseback,
                wch a Staff in his hand, & this motto: Max. Eman. DG. V. Ba. TE. P. S. D. C. P. R. S.
                R. J. ar. Te. L. L.
            
              17.
            
          
Duke of Bavaria
D. C
247

div3
Impressions of Seals &c.
EPF108491230

            249
            A Cast of the Coronation medal of Frid. 3. k. of Prussia.
              His Bust supported by Pallas, & Plenty with her horn. The Inscription at top:
                nec virtuti gloria deest. and at Bottom: Frideric. III. D. G. Brand. Supr. Borussiœ
              Prin. in Regem inauguratus D. 18. Jan: 1701.*..
            
              17.
            
          
Prussia.
249

div3
Impressions of Seals &c.
EPF108491230

            250.
            Prussia, in ye form of a
              Queen sitting, wth an angel behind sounding the Trump of
              Fame, wth this Motto at Top.
              Restituit nomenque decu

Valerius Bellus
299

div3
Pictures &c.
EPF108491264

            -301.
             Sr. Richard Baker author of the History of England in wax.
            
              Collect Min 
              ✓
              Va.
            
          
Sr. Richard Baker
England
301

div3
Pictures &c.
EPF108491264

            -304. 305
            Christ taken y down from the
              Crosse.  Valerius
                Bellus Vicent. F. a lead Basso relievo.
            
              Collect
              C.H.R.
            
          
Valerius Bellus
304 305

div3
Pictures &c.
EPF108491264

            -306.
            Ecce agnus dei a picture made of feathers. From  Mr. Dupuys.
            
              Collect ✓
            
          
Mr. Dupuys.
306

div3
Pictures &c.
EPF108491264

            -307.
            Two playing at draughts after 
                Callot painted in Silk. Id. 
            
              Collect ✓
              vid. Gamesters.
            
          
Callot
307

In [11]:
with open('people_and_places.csv', 'w') as file:
    writer = csv.writer(file)
    for row in people_csv_data:
        writer.writerow(row)